In [30]:
import csv
import io
import re
import string
import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
data = pd.read_csv("C:/Users/rober/Desktop/Answers.csv", encoding='latin1')
data.head()

ModuleNotFoundError: No module named 'tqdm'

In [3]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8


In [5]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [6]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [7]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


In [8]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))

26


In [9]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(5, 4): (in, shimmered)
(2, 4): (wide, shimmered)
(3, 5): (road, in)
(6, 7): (hot, sun)
(4, 3): (shimmered, road)


In [28]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([170  19 826  57], shape=(4,), dtype=int64)
['pand', 'be', 'box', 'want']


In [27]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word

In [20]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 3
target_word     : road
context_indices : [2 2 1 4 3]
context_words   : ['wide', 'wide', 'the', 'shimmered', 'road']
label           : [1 0 0 0 0]


In [21]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : 3
context : tf.Tensor([2 2 1 4 3], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

In [25]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [26]:
print(data.head(20))
file_path = "C:/Users/rober/Desktop/Answers.csv"
with open(file_path, 'r', encoding='latin1') as f:
    lines = f.read().splitlines()
for line in lines[:20]:
    print(line)

     Id  OwnerUserId  ... Score                                               Body
0    92         61.0  ...    13  <p><a href="http://svnbook.red-bean.com/">Vers...
1   124         26.0  ...    12  <p>I wound up using this. It is a kind of a ha...
2   199         50.0  ...     1  <p>I've read somewhere the human eye can't dis...
3   269         91.0  ...     4  <p>Yes, I thought about that, but I soon figur...
4   307         49.0  ...    28  <p><a href="http://www.codeproject.com/Article...
5   332         59.0  ...    19  <p>I would be a bit reluctant to use nested cl...
6   344        100.0  ...     6  <p>You might be able to use IronRuby for that....
7   359        119.0  ...     5  <P>You could use any of the DLR languages, whi...
8   473         49.0  ...     8  <p>No, what you're doing is fine. Don't let th...
9   529         86.0  ...     3  <p>Isn't it also a factor which order you set ...
10  539        157.0  ...    21  <p>My first thought on this is "how generate N...
11  

In [12]:
# Call the data dataframe:
df = pd.DataFrame(data)
# Function to check for links in q text
def has_link(text):
    link_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return not bool(link_pattern.search(text))
# Extract the "Body" column and filter out rows with links
filtered_body_column = df[df['Body'].apply(has_link)]['Body']
# Create a TextLineDataset from the filtered "Body" column
ds_filtered = tf.data.TextLineDataset.from_tensor_slices(filtered_body_column)

In [13]:
# Remove punctuation
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [15]:
vectorize_layer.adapt(ds_filtered.batch(1024))

In [16]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'to', 'you', 'a', 'is', 'and', 'in', 'codepre', 'of', 'it', 'that', 'your', 'for', 'if', 'this', 'i', 'as', 'be']


In [17]:
# Vectorize the data in text_ds.
text_vector_ds = ds_filtered.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [19]:
# Obtain Sequence from the dataset
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

1304303


In [20]:
#Inspect a few examples from sequences:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 59   1 128  46  16  11   6   5 672  10] => ['pi', '[UNK]', 'up', 'using', 'this', 'it', 'is', 'a', 'kind', 'of']
[1053  234 1069    2 3740    1  221    1  244  595] => ['pive', 'read', 'somewhere', 'the', 'human', '[UNK]', 'cant', '[UNK]', 'between', 'less']
[ 830   17 1379  159   12   36   17 1732 1536  117] => ['pyes', 'i', 'thought', 'about', 'that', 'but', 'i', 'soon', 'figured', 'out']
[  44   81   26   90   10    2    1 1390   50  594] => ['pyou', 'could', 'use', 'any', 'of', 'the', '[UNK]', 'languages', 'which', 'provide']
[900  70 132 251   6 612  83 328 262 748] => ['pno', 'what', 'youre', 'doing', 'is', 'fine', 'dont', 'let', 'those', 'people']


In [26]:
# Generate training examples from sequences
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

NameError: name 'tqdm' is not defined